In [1]:
from cytoreactors.operate import operate
from cytoreactors.design.program import TurbidostatProgram, GrowDiluteProgram, Preculture
from cytoreactors.design.events import Event
from matplotlib import pyplot as plt
from time import sleep, time
from datetime import date
from threading import Timer
import pandas as pd
import numpy as np
import os

## Experiments notes

* ...

## Hardware checklist

### Temperature control

* **circulation pump**: should be plugged AND working. If ON but not working, unplug, aspirate from eand of circuit to prime the circuit using syringe, and plug again

* **water bath heating**: just turn on via the power button on the back. normally no need to use any of the face button. check the set temp is 31.7 AND that the water bath is heating. if there is only one temperature displayed IT IS NOT ON. In that case press the on button on the face panel 

### Aeration

* check or put the air input pressure around the arrow

* check if the air input line is not disconnected at the level of the big filter or before

### Reactor machine

* check that it is on: red LED on the right ?

* if off, the switch is located on the back, left side, bottom

### LEDs control

* test that the LED service is on: http://localhost:5000 (should give hello message)
* **IF NOT RESTART OF ONGOING PROGRAMS**
    - test that works: 
         + http://localhost:5000/turn_on_all/50
         + http://localhost:5000/turn_off_all
    - reset history of the service: http://localhost:5000/reset_history
* if restart, do nothing, in order to keep the history of the service and the ongoing duty cycling
      

### Robot and cytometer

* clean guava, with empty round-bottom plate + tubes as for acquisition (start worklist and cancel to know which tubes), screen on main acquisition window

* check the guava app working: http://157.99.244.170:5000/start_atlas_service should return 'Atlas service already running'

* check the guava clicking is working: http://157.99.244.170:5000/prepare/A12_B12/5000 should create the acquisition  worklist and start to run it until the tray is opening. Then click cancel and load the tray

* robot physical setup: new tips on the two boxes, clean full 12-row through, two empty round-bottom plates for sampling (super careful with positioning, ask Francois or Achille first), collector for tips liquid at the right of funnel, put some bleach via waste funnel to remove clogs in case. NO TIPS ON THE ARM.

* robot software setup
     - if flask app is running and there is tips on the pipette, drop them: ( **dont click if physical setup not good !**): http://169.254.6.20:5000/tips/drop
     - new exp if new exp or you renewed tip boxes, restart the flask app (see below). REMOVE TIPS MANUALLY IF THERE ARE LEFT ON PIPETTE
     - you can check that the robot app works by telling him to go the bin : http://169.254.6.20:5000/gotobin

* To restart the robot flask app:
     - go the the robot jupyter notebook server: http://169.254.56.89:48888/tree?
     - in the tab running check the terminals, normally should be one (or zero if you had to restart the robot)
     - if no terminal: 
          + create one, enter `cd /var/lib/jupyter/notebooks`, 
          + IMPORTANT FOR THE LOGs: enter the date via entering `date -s "2020-10-09 10:30"` WITH TWO HOURS LESS THAN REAL TIME BECAUSE UTC !
          + start the app: sh start_app.sh
     - if terminal:
          + ctrl+C to kill current process
          + type `date` to check the date, change if needed
          + start the app: sh start_app.sh

### Vessels and connections

* **GOOD ORIENTATION OF EACH VESSEL !!!! the vessel number should face you. critical for OD. Also, is the bubbling metal tube align with the mark on the glass vessel ?????**

* tape-secured drain tubing in pinch-valves

* check height of metal tubing in vessels using the marking standard metal tube

* no tubing too much on tension? happens often for media tubing

### Waste tank

* check waste tank is empty enough, otherwise change


## Defining the experiment configuration

In [2]:
# define the strains
precult_1 = Preculture(strain_id='IB20337 & IB20175',
                          media='SC LoFlo 2% glucose + 5mM Arginine',
                          strain_name='Sensor & PL_Nanobodies')

In [3]:
# create the programs
programs = {}

for rid in [1,2,3,4,5,6,7,8]:    
    programs[rid] = TurbidostatProgram(
        user = 'Sara',
        campaign = 'Debug',
        short_name = 'FlaskApp_3',
        description = 'Clicking software got crazy',
        reactor_id = rid,
        preculture = precult_1,
        media = 'SC LoFlo 2% glucose + 5mM Arginine',
        position_drain_output_mL = 20, # or 30 
        OD_setpoint=0.5,
        vessel_id=0, # can be useful ! replace by real id
        blank=None, # write a value only if absolutely necessary
        creation_date_shift=1) # change to 1 or 2 if restart of experiment done before than today

**CAREFUL: if doing a restart, you should have a confirmation dialog for each reactor at the step above**. If this not the case and it should, you probably did not set the *creation_date_shift* to 1 or 2 if restarting an experiment start 1 or 2 days earlier. If this happens please delete the useless folders on ATLAS.

## Starting a session

The next cell creates a bioreactor session, i.e. establish connections to hardware and starting a log.

In [4]:
#to say if you use the robot or not (fake_OT2) and if automation with beads
operate.session_created = False
session = operate.Session(fake_OT2=False, automated_beads = False)
print(session.status)

Logging filepath = C:\Users\77Z14G2\InbioReactors\cytoreactors/logs/operation_2023-02-23_1677149779.log
created


In [ ]:
# OPTIONAL ! ACTIVATE NO SAMPLING AT LOW VOLUME SAFETY
# WARNING: if weird vessel design (reservoir etc) the volume estimation will be wrong and this will cause issues
session.low_volume_no_sampling_safety = True

A session should be loaded with `Programs`, with one program per reactor.

This is done in the next cell.

In [5]:
#to load the program
for program in programs.values():
    session.add_program(program)

Note that we haven't started the session yet, so no main loop is running.

## Vessel setup

In [12]:
session.pinch_drain_valves()

In [11]:
session.unpinch_drain_valves()

## Pump registration

Define all pumps id is needed to use pump-specific flow rate from calibration data !

In [6]:
# input pumps
session.manager.input_pumps[1] = 'M1-3-2'
session.manager.input_pumps[2] = 'M3-1-1'
session.manager.input_pumps[3] = 'M6-1-2'
session.manager.input_pumps[4] = 'M4-3-1'
# output pumps
session.manager.output_pumps[1] = 'H1'
session.manager.output_pumps[2] = 'H2'
session.manager.output_pumps[3] = 'H3'
session.manager.output_pumps[4] = 'H4'

In [7]:
# input pumps
session.manager.input_pumps[5] = 'M4-2-3'
session.manager.input_pumps[6] = 'M4-1-1'
session.manager.input_pumps[7] = 'M6-2-2'
session.manager.input_pumps[8] = 'M4-4-1'
# output pumps
session.manager.output_pumps[5] = 'H5'
session.manager.output_pumps[6] = 'H6'
session.manager.output_pumps[7] = 'H7'
session.manager.output_pumps[8] = 'H8'

## Testing bubbling and stop of bubbling  

In [13]:
for valve_id in ['B1','B2','B3','B4']:
    session.manager.open_valve_for_duration(valve_id,10)
sleep(10)

## Priming pumps, adding media

When physically setting up vessels, there is no media in the vessels, and the tubing lines feeding vessel with media are filled with air.

Priming pumps is needed to establish liquid into the media feed lines.

**please use this opportunity to flag problems with the media lines !**

In [14]:
#to put a bit of media in the vessels
session.prime_input_pumps()

Priming of pumps with liquid. Make sure pump output is setup as desired. Enter y to flow for 6.000000 seconds, otherwise ok
y
Priming of pumps with liquid. Make sure pump output is setup as desired. Enter y to flow for 6.000000 seconds, otherwise ok
ok


The priming has left some media in the vessels, plus usuqlly some water after autoclaving, so better to remove it.

Assuming sampling tubing is to the bottom, we will open them.

In [ ]:
# safety !
session.manager.send_ot2_request('gotobin')

In [15]:
session.pinch_drain_valves()
session.start_output_flow()

In [16]:
session.stop_output_flow()
session.unpinch_drain_valves()

Use the standard stick for setting drain tube height (20 or 30 mL).

**Use the standard stick for setting sampling tube height back up**

Then adapt desired vol below and add media by running the cell.

In [ ]:
# this work only for same volume in all vessels for now...
volume_vessel_mL = list(session.programs.values())[0].start_volume_mL
session.add_media(volume_mL=volume_vessel_mL)

Now we can start the session.

It will loop (OD measurements mainly).

The first readings will be considered as blank, so **don't inoculate before blank is finished.**

You will know when blanking is finished by **running the OD plotting cell** (see below): if the plot is empty, it means blanking phase not finished.

Note that the plots don't update themselves, you have to re-execute the cell.

## Starting the session

In [17]:
session.start()
session.status

Starting !


'running'

In [ ]:
session.status

## Defining events

In [ ]:
## removing existing events in case
for program in session.programs.values():
    program.events = []

In [ ]:
from cytoreactors.control.MPC import trigger_new_cyto_data
import logging

def action_UPR_ramp_feedback(program, pars, state):
    # extract last cytometer data
    last_tp = state['current_tp']
    data_last_tp = program.cells[program.cells['time_s'] == last_tp].copy()
    
    # Process data befor event check
    # Apply corrections
    correction_coef_ORG_G = 1.35
    
    data_last_tp['ORG-G_norm'] = data_last_tp['ORG-G-HLin'] * correction_coef_ORG_G    
            
    # Gate by size
    data_last_tp_gated_size = data_last_tp[(data_last_tp['FSC-HLin'] > 2*10**3) & \
                                   (data_last_tp['FSC-HLin'] < 3*10**3)].copy()
    
    # Normalize to size
    data_last_tp_gated_size['GRN-V/FSC'] = data_last_tp_gated_size['GRN-V-HLin']/data_last_tp_gated_size['FSC-HLin']
    data_last_tp_gated_size['ORG-G/FSC'] = data_last_tp_gated_size['ORG-G_norm']/data_last_tp_gated_size['FSC-HLin']
    
    # Gate from sensor
    data_last_tp_gated_from_sensor = data_last_tp_gated_size[data_last_tp_gated_size['GRN-V/FSC'] < 0.05].copy()
    
    # compute the UPR metric
    UPR_level = data_last_tp_gated_from_sensor['ORG-G/FSC'].mean()
    
    # changing light based on value
    # should we increase the duty cycle ?
    if UPR_level < pars['threshold']:
        state['current_dc'] += pars['DC'] 
        if state['current_dc'] > 1:
            state['current_dc'] = 1
    # should we decrease the duty cycle ?
    if UPR_level > pars['threshold']:
        state['current_dc'] -= pars['DC'] 
        if state['current_dc'] < 0:
            state['current_dc'] = 0
            
    logging.info('T= {}| UPR feedback control reactor {}: UPR level = {}, new dc = {}'.format(time(),program.reactor_id,UPR_level,state['current_dc']))
    
    # apply current dc
    # start_LED_duty_cycle(intensity, period_s, fraction, n_cycles)
    program.start_LED_duty_cycle(20, 45*60, state['current_dc'], 2000)
    
    # update event state
    state['last_change_tp'] = state['current_tp']

In [ ]:
 # different windows of UPR
targets_UPR_windows_and_DC = {1:(0.13,0.1), 2:(0.13,0.1), 3:(0.15,0.1), 4:(0.15,0.1), 5:(0.17,0.1), 6:(0.17,0.1), 7:(1,1), 8:(1,1)}

In [ ]:
for rid in [1, 2, 3, 4, 5, 6, 7, 8]:
    thresh, DC = targets_UPR_windows_and_DC[rid]
    event_UPR = Event(trigger=trigger_new_cyto_data,
                      action=action_UPR_ramp_feedback,
                      state={'last_change_tp':0, 'current_tp':0, 'current_dc':0.},
                      pars={'threshold':thresh, 'DC':DC})
    session.programs[rid].events.append(event_UPR)

## Inspecting program data

The main loop is living in a thread, so you can live-inspect program data in this notebook.

* The first cell below is plotting blanks measurements, 
* the two after the OD measurements (what comes after the blank is done)

Execute those cells as many times as you wish.

In [ ]:
# reset blanks
for program in programs.values():
    program.blank = None
    program.blanks = {'time_s':[], 'blank_OD':[]}

In [ ]:
# extract data as dataframe for each reactor
dfs = {prog.reactor_id:prog.data_to_df('blanks') for prog in session.programs.values()}
# loop on reactors
plt.figure(figsize=(16,8))
for rid,df in dfs.items():
    color = plt.cm.jet(np.linspace(0,1,16))[rid-1]
    if not df.empty:
        plt.plot((df['time_s']-df['time_s'][0])/3600., df['blank_OD'],'-o',color=color)
    else:
        print('No data yet')
        break
    # styling
plt.legend(dfs.keys())
plt.xlabel('Time (hrs)')
plt.ylabel('single blank measure')
plt.show()

In [ ]:
# show OD, growth rate, LED data for all reactors
ax_OD, ax_gr, ax_LEDs = session.plot_reactor_group(show_fl=False, reactor_ids=[1, 2, 3, 4, 6, 7, 8])
ax_OD.set(xlim=(0,24))
ax_OD.set(ylim=(0,1))

In [ ]:
session.status

In [ ]:
# same but showing cytometry data as well (median, ungated)
ax_OD, ax_gr, ax_LEDs, ax_fl = session.plot_reactor_group(show_fl=True, ch_fl='GRN-B-HLin', reactor_ids=[1,2,3,4,6,7,8])
ax_fl.set_ylim([0,10*10**2])
ax_OD.set_ylim([-0.05,1])
ax_OD.set_xlim([0,24])

In [ ]:
#FSC = size --> to see if conta
for i in [1,2,3,4,6,7,8]:
        session.programs[i].plot_all_cells(ch_fl='FSC-HLin')

In [ ]:
#check mCer (sensor)

for i in [1,2,3,4,6,7,8]:
        session.programs[i].plot_all_cells(ch_fl='GRN-B-HLin')

## Inoculation

When blanking phase is finished, you might want to inoculate.

**DON'T INOCULATE BEFORE END OF BLANKING !!!!!!!**

**After, let's pump out via the drain this time to remove excess volume.**

In [ ]:
session.pause()
session.start_output_flow()

In [ ]:
session.stop_output_flow()
session.start()

## Changing program parameters: OD setpoint, LED intensities, etc...

Because the main loop is running in a thread, you have access to the `Program` object, so you can live modify them.

Modifications will be 'seen' by the main loop thread.

In [ ]:
# display current OD setpoint and OD
for program in session.programs.values():
    print(f'{program.reactor_id} - {program.OD_setpoint} - {program.give_last_OD()}')

In [ ]:
for program in session.programs.values():
    program.OD_setpoint = 1.0

In [ ]:
# set LED intensities. 0 is dark, 255 is max intensity. It erases oingoing duty cycling !
# now this cell can take some time (32 seconds...)
intensities = {rid:50 for rid in range(1,17)}
for rid,intensity in intensities.items():
    program = session.programs[rid]
    program.set_LED(intensity)

In [ ]:
# set LED duty cycling
period_s = 0.5*3600
n_cycles = 200
intensity = 40
duty_cycles = {9:0, 10:0.25, 15:0.5, 16:0.75}
for rid,fraction in duty_cycles.items():
    program = session.programs[rid]
    program.start_LED_duty_cycle(intensity, period_s, fraction, n_cycles)

In [ ]:
# use a timer to change something in a certain amount of time
def change_od_setpoint():
    for program in session.programs.values():
        program.OD_setpoint = 1.2
t = Timer(2.5 * 3600, change_od_setpoint)
t.start()

In [ ]:
# set LED duty cycling
period_s = 48*3600
n_cycles = 2000
intensity = 20
duty_cycles = {1:0.125, 8:0.125}
for rid,fraction in duty_cycles.items():
    program = session.programs[rid]
    program.start_LED_duty_cycle(intensity, period_s, fraction, n_cycles)

In [ ]:
# set LED duty cycling
period_s = 48*3600
n_cycles = 2000
intensity = 20
duty_cycles = {2:1, 3:1, 4:1, 5:1, 6:1, 7:1}
for rid,fraction in duty_cycles.items():
    program = session.programs[rid]
    program.start_LED_duty_cycle(intensity, period_s, fraction, n_cycles)

In [ ]:
# set LED duty cycling
period_s = 0.5*3600
n_cycles = 2000
intensity = 20
duty_cycles = {1:0, 2:0.15, 3:0.3, 4:0.85, 6:0.15, 7:0.43, 8:1}
for rid,fraction in duty_cycles.items():
    program = session.programs[rid]
    program.start_LED_duty_cycle(intensity, period_s, fraction, n_cycles)

In [8]:
# set LED duty cycling
#From former exp of Sebas
period_s = 0.5*3600
n_cycles = 2000
intensity = 20
duty_cycles = {1:0, 2:0.143, 3:0.35, 4:0.45, 5:0.571, 6:0.714, 7:0.857, 8:1}
for rid,fraction in duty_cycles.items():
    program = session.programs[rid]
    program.start_LED_duty_cycle(intensity, period_s, fraction, n_cycles)

## Scheduling cytometry

removed outdated info....

In [18]:
## activate cytometry measurements for reactors 1-8
for rid in [1,2,3,4,5,6,7,8]:
    session.programs[rid].active_cytometry = True

In [19]:
## display active reactors for cytometry
session.programs_with_cytometry

{1: <cytoreactors.design.program.TurbidostatProgram at 0xb94c5c8>,
 2: <cytoreactors.design.program.TurbidostatProgram at 0x9149688>,
 3: <cytoreactors.design.program.TurbidostatProgram at 0x561d908>,
 4: <cytoreactors.design.program.TurbidostatProgram at 0xb94c948>,
 5: <cytoreactors.design.program.TurbidostatProgram at 0x53b3d08>,
 6: <cytoreactors.design.program.TurbidostatProgram at 0x560c2c8>,
 7: <cytoreactors.design.program.TurbidostatProgram at 0x561de48>,
 8: <cytoreactors.design.program.TurbidostatProgram at 0xb94c888>}

In [ ]:
## to use if you want to renew volume via media input after sampling
for rid in [1,2,3,4,5,6,7,8]:
    session.programs[rid].renew_sampled_volume = False

In [47]:
## scheduling a SINGLE timepoint NOW
session.schedule_sampling(time())

too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None


In [ ]:
## scheduling with 15 hours delay
for i in range(23):
    session.schedule_sampling(time()+12.25*3600+i*45*60)

In [ ]:
## scheduling a SINGLE timepoint in 2 hours from now
session.schedule_sampling(time()+2*3600)

In [49]:
## scheduling 12 timepoints separated by 45mins starting now
for i in range(24):
    session.schedule_sampling(time()+i*15*60)

In [ ]:
session.samplings[-1]

In [ ]:
session.sampling_schedule = []
last_tp = session.samplings[-1]
for i in range(24):
    session.schedule_sampling(last_tp+(i+1)*60)

In [52]:
## display if we are waiting for guava data
session.waiting_for_guava_data 

True

In [54]:
session.waiting_for_guava_data = False

In [53]:
## display if wash ongoing or about to start
session.should_wash

False

too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None
too few events in the sample, returning None


In [ ]:
## display the number of done samplings
len(session.samplings)

In [46]:
session.sampling_schedule = []

In [51]:
## display the sampling schedule
for tp in session.sampling_schedule:
    dt_min = (tp-time())/60.
    print(f'Timepoint in {dt_min} minutes')

Timepoint in -22.08408525387446 minutes
Timepoint in -7.084085253874461 minutes
Timepoint in 7.915914746125539 minutes
Timepoint in 22.91591474612554 minutes
Timepoint in 37.91591474612554 minutes
Timepoint in 52.91591474612554 minutes
Timepoint in 67.91591474612554 minutes
Timepoint in 82.91591474612554 minutes
Timepoint in 97.91591474612554 minutes
Timepoint in 112.91591474612554 minutes
Timepoint in 127.91591474612554 minutes
Timepoint in 142.91591474612554 minutes
Timepoint in 157.91591474612554 minutes
Timepoint in 172.91591474612554 minutes
Timepoint in 187.91591474612554 minutes
Timepoint in 202.91591474612554 minutes
Timepoint in 217.91591474612554 minutes
Timepoint in 232.91591474612554 minutes
Timepoint in 247.91591474612554 minutes
Timepoint in 262.91591474612557 minutes
Timepoint in 277.91591474612557 minutes
Timepoint in 292.91591474612557 minutes
Timepoint in 307.91591474612557 minutes


The following cell is to call ONLY after replacing full state sampling

* you should have new tips, new plates, reservoir in position
* it will ask you to WIPE any drops on the sampling output lines
* if the session is running, it will pause it
* it will ask for your 'ok' when you have done it
* it will then move above both sampling plates to check correct positioning
* if the session was running, it will restart it

In [ ]:
## READ ABOVE
session.reset_all_sampling_state()

In [ ]:
## display the current plate and current col
print(f'plate num: {session.ot2_plate_num} -- column: {session.ot2_col}')

## Pausing a session, restarting a session

Pausing a session can be useful to start a sampling program, or to inspect the vessel, etc...

**DON'T FORGET TO START BACK**

In [ ]:
print(session.status)

In [ ]:
session.stop()
print(session.status)

In [ ]:
session.pause()

In [ ]:
session.start()
print(session.status)

## Taking samples manually

In [ ]:
session.pause()
session.status

In [ ]:
session.pinch_drain_valves()

In [ ]:
# dead volume
for rid in [1,2,3,4,6,7,8]:
    session.manager.open_pump_for_volume(f'H{rid}', 1)

In [ ]:
# to do one by on
rid = 8
session.manager.open_pump_for_volume(f'H{rid}', 1)
print(f'done {rid}')

In [ ]:
session.unpinch_drain_valves()

In [ ]:
session.start()
session.status

In [ ]:
(24 - (time()-session.programs[2].LEDs['time_s'][0])/3600)

## Stopping a run and cleaning procedure

In [55]:
# if can stop everything
session.stop()
print(session.status)

Stopping session
stopped


In [ ]:
# copy cyto data to atlas
for program in session.programs.values():
    program.cells.to_csv(f'{program.atlas_path}/cells.csv', index=False)

In [ ]:
# if don't want to stop because not cleaning all reactors (for instance cleaning 1-8 when 9-16) active
# then OK not to STOP EXCEPT IF CYTOMETRY TIMEPOINTS TO COME DURING THE CLEANING !!!!
# the method below will also remove corresponding programs from main loop
session.register_reactors_for_cleaning([1,2,3,5,4,6,7,8])

### 1. remove bulk of culture

Put both drain and sampling metal tubing DOWN.

In [ ]:
# now we open outputs with a volume of 35 mL per reactor to make sure we removed everything
session.remove_volume(volume_mL=25, cleaning=True, pinch_drain_valves=False)

In [ ]:
# the cell above should have been enough to have removed everything (i.e. mostly air coming out)
# IF NOT THE CASE, use this cell to run a bit more
session.remove_volume(volume_mL=5, cleaning=True, pinch_drain_valves=False)

### 2. recover media bottles

If 'decent' amount of media left in some of the media bottles, you can recover them by rapidly closing them with a cap from a autoclaved blue cap empty bottle.

Swab the the input metal tubings with tissue paper because it is wet with media.

### 3. Running bleach through the system

For 8 reactors, prepare a 500 mL becher with 200 mL (25 mL per reactor) of 0.5% bleach (~5 times dilution from the bleach bottle).

Position the 2x4 input metal tubings in the becher.

In [ ]:
# we add first ~ half of it
session.add_media(volume_mL=35, cleaning=True)

We will now add 25 mL more BUT running through the inoculation port to clean it !

Connect the input pump output tubing to the INOCULATION PORT (with keck open obviously)

In [ ]:
# we add the remainder (through the inoculation port)
session.add_media(volume_mL=25, cleaning=True)

Now we have about 25 mL (level cannot be seen normally) of bleach in the vessel.

Will we run all of it from both output lines.

In [ ]:
session.remove_volume(volume_mL=25, cleaning=True, pinch_drain_valves=True)
session.remove_volume(volume_mL=30, cleaning=True, pinch_drain_valves=False)

In [ ]:
session.remove_volume(volume_mL=15, cleaning=True, pinch_drain_valves=False)

### 4. Running how water through the system

For 8 reactors, prepare a 500 mL becher with 400 mL ( mL per reactor) of hot (microwave for 45 secs) DI water.
Position the 2x4 input metal tubings in the becher.

In [ ]:
# we add first ~ half of it (through the inoc port)
session.add_media(volume_mL=20, cleaning=True)

Connect back the input pump output tubing to the media input port AND CLOSE THE KECK OF INOCULATION PORT

In [ ]:
# we add the remainder (through the media input port)
session.add_media(volume_mL=20, cleaning=True)

We should now be at ball level ! Let's wait a bit, an let's go down a bit below the ball for next step, whi is output pump calib. this also prime the output lines for the calib.

In [ ]:
# going below ball and output lines priming
session.remove_volume(volume_mL=10, cleaning=True, pinch_drain_valves=False)

### 5. Output pump calibration

Position the pump calibration device on the deck (at the place of the the first sampling plate holder), with the label facing you.

Normaly, already primed output lines.

In [ ]:
# IF session running because other reactors, we should pause for calibration
session.pause()

In [ ]:
# moving above measurement device
session.manager.send_ot2_request('move_to/sampling_metal/2')

Now, ONE BY ONE, we will CALIBRATE by turning on and then MANUALLY TURNING OFF each output pump UNTIL THE MEASUREMENT DEVICE ROW IS FULL.

RUN THE NEXT CELL ONLY WHEN YOU ARE READY TO PRESS ENTER EVERYTIME YOU FILLED THE MEASUREMENT DEVICE !

In [ ]:
session.start_output_pump_calibration()

In [ ]:
session.manager.send_ot2_request('gotobin')

### 6. Input pump calibration

Empty completely the measurement device, and put it back in place.

ADD water to becher (150 for 8 reactors)

Connect the output of INPUT pumps to the sampling lines

In [ ]:
# prime sampling lines
session.add_media(volume_mL=5, cleaning=True)

In [ ]:
# moving above measurement device
session.manager.send_ot2_request('move_to/sampling_metal/2')

Now, ONE BY ONE, we will CALIBRATE by turning on and then MANUALLY TURNING OFF each input pump UNTIL THE MEASUREMENT DEVICE ROW IS FULL.

RUN THE NEXT CELL ONLY WHEN YOU ARE READY TO PRESS ENTER EVERYTIME YOU FILLED THE MEASUREMENT DEVICE !

In [ ]:
session.start_input_pump_calibration()

In [ ]:
session.manager.send_ot2_request('gotobin')

In [ ]:
# IF session running because other reactors, we can resume here
session.start()

NOW CONNECT BACK SAMPLING LINES TO OUTPUT PUMPS AND OUTPUT OF INPUT PUMPS TO NORMAL VESSEL INPUTS !

### 7. Dry input lines and remove all water via both output lines

In [ ]:
input('ARE YOU SURE ????????????? OUTPUT PUMP CONNECTED TO OUTPUT LINES ????????????')
session.add_media(volume_mL=5, cleaning=True)
session.remove_volume(volume_mL=15, cleaning=True, pinch_drain_valves=True)
session.remove_volume(volume_mL=15, cleaning=True, pinch_drain_valves=False)
session.remove_volume(volume_mL=5, cleaning=True, pinch_drain_valves=True)

While this is happening, you can disengage back of pinch valves and OPEN KECK VALVES IF BELOW BALL

### 8. disengage tubing from pinch valves

first disengage drain tubing, then pinch, disengage sampling, then unpinch

In [ ]:
session.pinch_drain_valves(reactor_ids=session.reactor_ids_to_clean)

In [ ]:
session.unpinch_drain_valves(reactor_ids=session.reactor_ids_to_clean)

### 9. Only if no other programs running of course 

* stop water bath and circulation
* guava clean
* thrash tips, clean reservoir
* thrash sampling plates
* clean all ot2 stuff
* check tank
* stop the valves and coils and leds

In [56]:
# safety
session.manager.shut_off_all_pumps_and_valves()

In [57]:
# shut of leds (CAREFUL: NOT LOGGED in program data)
session.manager.shut_off_all_LEDs()